In [30]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [31]:
if torch.cuda.is_available():
    device=torch.device(type="cuda",index=0)
else:
    device=torch.device(type="cpu",index=0)

In [32]:
class WineDataset(Dataset):
    def __init__(self, transform=None):
        xy = np.loadtxt('winequality-red.csv', delimiter=",", dtype=np.float32, skiprows=1)
        self.x = xy[:, 1:]  # All features (inputs)
        self.y = xy[:, 0].astype(np.int64)  # Targets (wine quality), convert to int64
        # Convert wine quality to class indices (5 -> 0, 6 -> 1, 7 -> 2)
        self.y = np.where((self.y >= 5) & (self.y <= 7), self.y - 5, 2)  # Map 5-7 to 0-2, others to 2
        self.n_samples = xy.shape[0]
        self.transform = transform

    def __getitem__(self, index):
        x_sample, y_sample = self.x[index], self.y[index]
        if self.transform:
            x_sample, y_sample = self.transform((x_sample, y_sample))
        return x_sample, y_sample

    def __len__(self):
        return self.n_samples

In [33]:
class ToTensor:
    def __call__(self, sample):
        inputs, targets = sample
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(targets, dtype=torch.long)

In [34]:
dataset = WineDataset(transform=ToTensor())
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [35]:
class FFNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.inp=nn.Linear(in_features=11,out_features=16)
        self.relu=nn.ReLU()
        self.bn1=nn.BatchNorm1d(num_features=16)
        self.h1=nn.Linear(in_features=16,out_features=16)
        self.bn2=nn.BatchNorm1d(num_features=16)
        self.h2=nn.Linear(in_features=16,out_features=3)
        self.bn3=nn.BatchNorm1d(num_features=3)
        

    def forward(self,x):
        x=self.inp(x)
        x=self.bn1(x) #16
        x=self.relu(x)
        x=self.h1(x)  
        x=self.bn2(x) #16
        x=self.relu(x)
        x=self.h2(x)
        output=self.bn3(x) # 3
        return output


In [36]:
model = FFNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [37]:
def train_epoch(dataloader, model, loss_fn, optimizer):
    model.train()
    total_loss = 0
    for i, (inputs, labels) in enumerate(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    return total_loss / len(dataloader)

In [38]:
def evaluate_model(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    return accuracy


In [39]:
n_epochs = 100
for epoch in range(n_epochs):
    loss = train_epoch(train_loader, model, loss_fn, optimizer)
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss:.4f}")

Epoch 1/100, Loss: 1.3100
Epoch 2/100, Loss: 1.0448
Epoch 3/100, Loss: 0.9872
Epoch 4/100, Loss: 0.9371
Epoch 5/100, Loss: 0.9046
Epoch 6/100, Loss: 0.8734
Epoch 7/100, Loss: 0.8327
Epoch 8/100, Loss: 0.7879
Epoch 9/100, Loss: 0.7532
Epoch 10/100, Loss: 0.7206
Epoch 11/100, Loss: 0.6969
Epoch 12/100, Loss: 0.6678
Epoch 13/100, Loss: 0.6508
Epoch 14/100, Loss: 0.6424
Epoch 15/100, Loss: 0.6266
Epoch 16/100, Loss: 0.6157
Epoch 17/100, Loss: 0.5790
Epoch 18/100, Loss: 0.5632
Epoch 19/100, Loss: 0.5486
Epoch 20/100, Loss: 0.5430
Epoch 21/100, Loss: 0.5340
Epoch 22/100, Loss: 0.5351
Epoch 23/100, Loss: 0.5251
Epoch 24/100, Loss: 0.5242
Epoch 25/100, Loss: 0.5147
Epoch 26/100, Loss: 0.5105
Epoch 27/100, Loss: 0.4970
Epoch 28/100, Loss: 0.4948
Epoch 29/100, Loss: 0.4788
Epoch 30/100, Loss: 0.4814
Epoch 31/100, Loss: 0.4675
Epoch 32/100, Loss: 0.4781
Epoch 33/100, Loss: 0.4620
Epoch 34/100, Loss: 0.4494
Epoch 35/100, Loss: 0.4571
Epoch 36/100, Loss: 0.4503
Epoch 37/100, Loss: 0.4372
Epoch 38/1

In [40]:
test_accuracy = evaluate_model(model, test_loader, device)
print(f"Accuracy on test set: {test_accuracy:.2f}%")

# You can also use this function to evaluate on the training set if needed
train_accuracy = evaluate_model(model, train_loader, device)
print(f"Accuracy on training set: {train_accuracy:.2f}%")

Accuracy on test set: 85.00%
Accuracy on training set: 87.18%
